<a href="https://colab.research.google.com/github/zahraDehghanian97/LSH_Spark/blob/master/Question3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# تمرین سری سوم درس پردازش داده های حجیم
# زهرا دهقانیان
# 401300417




In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq --fix-missing
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 15.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=8e8d792aa148268f4cc299d65e8ce3cc04f121da31b51a68859fb0ac35eb56ab
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The followin

In [2]:
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://security.ubuntu.com/ubuntu 

In [3]:
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
from pyspark import SparkContext, SparkConf 
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,TimestampType
from pyspark.sql.functions import col,current_timestamp,to_date,hour,dayofweek
spark = SparkSession \
    .builder \
    .appName("Fave_Project") \
    .master("local[*]") \
    .getOrCreate()

sc=spark.sparkContext

schema = StructType([ \
        StructField("DEVICE_CODE", IntegerType(), True), 
        StructField("SYSTEM_ID",IntegerType(),True), \
        StructField("ORIGINE_CAR_KEY",StringType(),True), \
        StructField("FINAL_CAR_KEY",StringType(),True), \
        StructField("CHECK_STATUS_KEY", IntegerType(), True), \
        StructField("COMPANY_ID", StringType(), True), \
        StructField("PASS_DAY_TIME", TimestampType(), True)
    ])

In [5]:
df=spark.read.csv('/content/drive/MyDrive/SampleData.csv',header=True,schema=schema)
df.show(1)

+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|DEVICE_CODE|SYSTEM_ID|ORIGINE_CAR_KEY|FINAL_CAR_KEY|CHECK_STATUS_KEY|COMPANY_ID|      PASS_DAY_TIME|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|  100700853|      283|        9016704|      9016704|               5|       264|2022-01-08 00:00:39|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+
only showing top 1 row



الف) مسیر عبوری هر خودرو را به تفکیک روز مشخص کنید 

In [6]:
df_mapped = df.rdd.map(lambda x: ((x['ORIGINE_CAR_KEY'], x['PASS_DAY_TIME'].date()), [x['DEVICE_CODE']]))
df_mapped3 = df_mapped.reduceByKey(lambda a,b: a+b)
df_mapped2 = df_mapped3.sample(False,0.3)
df_mapped2.take(10)

[(('9035020', datetime.date(2022, 1, 8)), [900164]),
 (('9021792', datetime.date(2022, 1, 8)),
  [900246,
   100700871,
   900235,
   631697,
   22009971,
   900265,
   900235,
   100700871,
   22009972,
   900215,
   209103]),
 (('9031414', datetime.date(2022, 1, 8)), [900191, 900129, 900275]),
 (('9031445', datetime.date(2022, 1, 8)), [100701092, 22010053]),
 (('9026982', datetime.date(2022, 1, 8)), [900266]),
 (('9160135', datetime.date(2022, 1, 8)), [230101]),
 (('9194030', datetime.date(2022, 1, 8)), [100701093, 900132]),
 (('9185571', datetime.date(2022, 1, 8)), [631488, 213401]),
 (('9164271', datetime.date(2022, 1, 8)), [900101, 900241, 900191]),
 (('9284980', datetime.date(2022, 1, 8)), [900259, 112, 200601, 900259])]

ب)محاسبه شباهت کسینوسی بین یک مسیر فرضی و مسیر خودروها و گزارش پنج مسیر و خودرو با بیشترین شباهت  

برای یافتن فاصله‌ی کسینوسی به جای تشکیل بردار با طول هزار متشکل از مقادیر صفر و یک برای هر مسیر میتوان تعداد کدهای مشترک بین هر مسیر و مسیر فرضی را بدست آورد و سپس بر اندازه‌ی بردار مسیر پیشنهادی و همچنین اندازه‌ی مسیر هر خودرو تقسیم نمود.
 در این مساله خروجی این روش همان خروجی موردنظر یعنی شباهت کسینوسی خواهد بود و فقط محاسبات ساده تر می شود

In [7]:
import math 
path = [900246,100700871,900235,631697] #masire farzi
lenPath = len(path)
similarity_rdd = df_mapped2.map(lambda x: ((x[0][0], x[1]), len(list(set(path) & set(x[1])))/math.sqrt(len(x[1])*lenPath)))
similarity_rdd.take(20)

[(('9035020', [900164]), 0.0),
 (('9021792',
   [900246,
    100700871,
    900235,
    631697,
    22009971,
    900265,
    900235,
    100700871,
    22009972,
    900215,
    209103]),
  0.38138503569823695),
 (('9031414', [900191, 900129, 900275]), 0.0),
 (('9031445', [100701092, 22010053]), 0.0),
 (('9026982', [900266]), 0.0),
 (('9160135', [230101]), 0.0),
 (('9194030', [100701093, 900132]), 0.0),
 (('9185571', [631488, 213401]), 0.0),
 (('9164271', [900101, 900241, 900191]), 0.0),
 (('9284980', [900259, 112, 200601, 900259]), 0.0),
 (('9326284', [22009923]), 0.0),
 (('9330082', [205802, 103001]), 0.0),
 (('9396599', [900222]), 0.0),
 (('9423518', [900101]), 0.0),
 (('9437711', [100701092, 900222, 900207]), 0.0),
 (('9435137', [900156, 22010079, 100701090, 900211, 900101]), 0.0),
 (('9582767', [900142, 900212, 100700853, 100700853, 100701151]), 0.0),
 (('9624394', [900207, 631363, 631775, 900225]), 0.0),
 (('9622673',
   [22010031,
    155,
    900218,
    155,
    631346,
    2

پس از بدست آمدن مقادیر شباهت کسینوسی هر مسیر با مسیر فرضی ۵ مسیر و خودرو با بیشترین شباهت با مسیر فرضی را نمایش میدهیم

In [8]:
similarPath = similarity_rdd.takeOrdered(5, key = lambda x: -x[1])
similarPath

[(('92669360', [900235, 900246, 100700871]), 0.5477225575051661),
 (('90645138', [631357, 900246, 100700871, 900235]), 0.4743416490252569),
 (('77158252', [900222, 900246, 900235, 100700871]), 0.4743416490252569),
 (('34991022', [100700871, 900235]), 0.4472135954999579),
 (('76678032', [100700871, 900235]), 0.4472135954999579)]

پ) با استفاده از روش ال اس اج بخش ب را حل کنید  و محاسبات مربوط به نحوه
استفاده از آن ها را انجام دهید. سپس مسیر های مشابه با مسیر فرضی را گزارش کنید. در صورت افزایش تعداد هایپرپلین ها دقت به چه صورت افزایش می یابد.

برای یافتن بردارهای مشابه با بردار فرضی تعریف شده به کمک روش ال اس اچ، اول بردار دوربینهای مجزا را می سازیم. در این مسیله ابتدا تعداد توابع مین هش یعنی هایپرپلین ها را برابر با 20۰ قرار میدهیم و در ادامه تغییرات را با دوبرابر کردنشان بررسی میکنیم. 

In [9]:
hyperplane = 20 
allcamera = df_mapped2.flatMap(lambda line: line[1])
allcameradist = allcamera.distinct().collect()
sc.broadcast(allcameradist)
l = len(allcameradist)
len(allcameradist)

912

 برای یافتن بردار هش شده‌ی به طول 20 از هر یک از مسیرها، بر اساس وجود اولین یک در هر یک از  جایگشت رندوم ایجاد شده در بالا، عمل می کنیم. در توابع زیرفرایند تولید جایگشت ها و بررسی وجود دوربین فوق در مسیر انجام میگیرد

In [10]:
import numpy as np
random_start = []
for i in range(hyperplane):
  random_start.append(np.random.permutation(l))
sc.broadcast(random_start)


In [16]:
x = [1]
for j in range(len(x)):
  print(x[j])

1


In [19]:

def findfirstone(arr, per, size):
  new_place = []
  for i in range(len(arr)):
    new_place.append(per[arr[i]])
  return min(new_place)

def minHashing(row):
  x = row[1]
  result = []
  place = []
  for j in range(len(x)):
      place.append(allcameradist.index(x[j]))
  for i in random_start:
      result.append(findfirstone(place, i, len(allcameradist)))

  return (row[1], result)

df_lsh = df_mapped2.map(minHashing)
df_lsh.take(1)

[([900164],
  [107,
   781,
   888,
   841,
   467,
   588,
   778,
   490,
   452,
   640,
   519,
   212,
   468,
   64,
   471,
   706,
   153,
   632,
   692,
   23])]

همین کار را برای مسیر فرضیمان نیز انجام میدهیم و بردار هش شده به طول 20 آن را بدست می‌آوریم

In [20]:
path = (0,[900246,100700871,900235,631697])
path_lsh = minHashing(path)
sc.broadcast(path_lsh)
path_lsh

([900246, 100700871, 900235, 631697],
 [205,
  137,
  35,
  188,
  103,
  188,
  23,
  231,
  3,
  159,
  37,
  395,
  51,
  497,
  9,
  366,
  480,
  30,
  156,
  47])

پس از بدست آوردن هش هر مسیر آن را به دو صورت با هش مسیر فرضی مقایسه میکنیم



1.   and then or
2.   or then and

در این جا خروجی برای هر مسیر به این صورت می باشد:

(تعداد هش های یکسان تقسیم بر کل تعداد هش ها، حاصل اور، حاصل اند،  مسیر )

r= 5 و b=4 

In [21]:
r = 5
b = 4
sc.broadcast(r)
sc.broadcast(b)

def check_similarity(x):
  y = x[1]
  orres = list(np.full(b, False))
  andres = list(np.full(b, True))
  sum = 0
  for i in range(len(y)):
    p = path_lsh[1][i] == y[i]
    andres[int(i/r)] = andres[int(i/r)] and p
    orres[int(i/r)] = orres[int(i/r)] or p
    if p:
      sum += 1
  return (x[0],(andres,orres,sum/len(y)))

df_similarity_LSH = df_lsh.map(check_similarity)
df_similarity_LSH.take(10)

[([900164], ([False, False, False, False], [False, False, False, False], 0.0)),
 ([900246,
   100700871,
   900235,
   631697,
   22009971,
   900265,
   900235,
   100700871,
   22009972,
   900215,
   209103],
  ([False, False, False, False], [True, True, True, True], 0.4)),
 ([900191, 900129, 900275],
  ([False, False, False, False], [False, False, False, False], 0.0)),
 ([100701092, 22010053],
  ([False, False, False, False], [False, False, False, False], 0.0)),
 ([900266], ([False, False, False, False], [False, False, False, False], 0.0)),
 ([230101], ([False, False, False, False], [False, False, False, False], 0.0)),
 ([100701093, 900132],
  ([False, False, False, False], [False, False, False, False], 0.0)),
 ([631488, 213401],
  ([False, False, False, False], [False, False, False, False], 0.0)),
 ([900101, 900241, 900191],
  ([False, False, False, False], [False, False, False, False], 0.0)),
 ([900259, 112, 200601, 900259],
  ([False, False, False, False], [False, False, False, 

In [35]:
def and_then_or(x):
    res = False
    temp = x[1][0]
    for i in temp:
      res = res or i
    return (x[0],(x[1][2],res))

and_or_result = df_similarity_LSH.map(and_then_or)
similar_or = and_or_result.filter(lambda x: x[1][1] == True)
similar_or.collect()

6


[([900235, 100700871, 900226, 900235, 900246], (0.6, True)),
 ([900174, 900174, 900235, 900246, 100700871], (0.6, True)),
 ([900174, 100700881, 900235, 900246, 100700871, 631357], (0.4, True)),
 ([631357, 900246, 100700871, 900235], (0.575, True)),
 ([900235, 209103, 900246, 100700871, 22010094, 22010088, 22004801],
  (0.475, True)),
 ([900235, 900246, 100700871], (0.725, True))]

در این حالت دو تا از پنج پرتکرارترین بازیابی شد

In [39]:
def or_then_and(x):
      res = True
      for i in x[1][1]:
        res = res and i
      return (x[0],(x[1][2],res))

or_and_result = df_similarity_LSH.map(or_then_and)
similar_and = or_and_result.filter(lambda x: x[1][1] == True)
similar_and.collect()

[([100700871], (0.3, True)),
 ([22010061, 100700871], (0.275, True)),
 ([100700871], (0.3, True)),
 ([100700871, 204101], (0.25, True)),
 ([100701119, 900167, 900266, 100700871, 209103, 135], (0.225, True)),
 ([119, 100700871, 900235], (0.375, True)),
 ([100701130, 631357, 100700871, 900235], (0.4, True)),
 ([100700871], (0.3, True)),
 ([100700871, 900149, 900274], (0.25, True)),
 ([100700871], (0.3, True)),
 ([100700871], (0.3, True)),
 ([100700804, 100700871, 900235, 209103], (0.4, True)),
 ([100700871], (0.3, True)),
 ([100700871], (0.3, True)),
 ([100700871, 900235], (0.525, True)),
 ([900266, 100700871], (0.275, True)),
 ([900162, 900183, 900235, 100700871, 22004601, 635719], (0.275, True)),
 ([100700871, 900235], (0.525, True)),
 ([100700871], (0.3, True)),
 ([100700871, 900235], (0.525, True)),
 ([100700871, 900142], (0.225, True)),
 ([900235, 100700871, 900226, 900235, 900246], (0.6, True)),
 ([900233, 22010060, 900149, 900233, 900233, 900149, 100700871], (0.2, True)),
 ([90023

بنظر می آید این روش چهارتا از پنج پرتکرار را یافته است

حال همین کار را با تعداد هایپر پلین های 40  انجام میدهیم

r=5 , b=8

In [25]:
hyperplane = 40
r = 5
b = 8
l = len(allcameradist)
sc.broadcast(r)
sc.broadcast(b)

import numpy as np
random_start = []
for i in range(hyperplane):
  random_start.append(np.random.permutation(l))
sc.broadcast(random_start)

df_lsh2 = df_mapped2.map(minHashing)
path_lsh = minHashing(path)
sc.broadcast(path_lsh)

df_similarity_LSH2 = df_lsh2.map(check_similarity)
df_similarity_LSH2.take(10)

[([900164],
  ([False, False, False, False, False, False, False, False],
   [False, False, False, False, False, False, False, False],
   0.0)),
 ([900246,
   100700871,
   900235,
   631697,
   22009971,
   900265,
   900235,
   100700871,
   22009972,
   900215,
   209103],
  ([False, False, False, False, False, False, False, False],
   [True, True, True, True, True, False, True, True],
   0.5)),
 ([900191, 900129, 900275],
  ([False, False, False, False, False, False, False, False],
   [False, False, False, False, False, False, False, False],
   0.0)),
 ([100701092, 22010053],
  ([False, False, False, False, False, False, False, False],
   [False, False, False, False, False, False, False, False],
   0.0)),
 ([900266],
  ([False, False, False, False, False, False, False, False],
   [False, False, False, False, False, False, False, False],
   0.0)),
 ([230101],
  ([False, False, False, False, False, False, False, False],
   [False, False, False, False, False, False, False, False],
   0

In [28]:
and_or_result2 = df_similarity_LSH2.map(and_then_or)

similar_or2 = and_or_result2.filter(lambda x: x[1][1] == True)
similar_or2.collect()

[([900235, 100700871, 900226, 900235, 900246], (0.6, True)),
 ([900174, 900174, 900235, 900246, 100700871], (0.6, True)),
 ([900174, 100700881, 900235, 900246, 100700871, 631357], (0.4, True)),
 ([631357, 900246, 100700871, 900235], (0.575, True)),
 ([900235, 209103, 900246, 100700871, 22010094, 22010088, 22004801],
  (0.475, True)),
 ([900235, 900246, 100700871], (0.725, True))]

بنظر می رسد این روش سه تا از پنج  شبیه ترین را یافته است

In [29]:
or_and_result2 = df_similarity_LSH2.map(or_then_and)

similar_and2 = or_and_result2.filter(lambda x: x[1][1] == True)
similar_and2.collect()

[([100700871], (0.3, True)),
 ([22010061, 100700871], (0.275, True)),
 ([100700871], (0.3, True)),
 ([100700871, 204101], (0.25, True)),
 ([100701119, 900167, 900266, 100700871, 209103, 135], (0.225, True)),
 ([119, 100700871, 900235], (0.375, True)),
 ([100701130, 631357, 100700871, 900235], (0.4, True)),
 ([100700871], (0.3, True)),
 ([100700871, 900149, 900274], (0.25, True)),
 ([100700871], (0.3, True)),
 ([100700871], (0.3, True)),
 ([100700804, 100700871, 900235, 209103], (0.4, True)),
 ([100700871], (0.3, True)),
 ([100700871], (0.3, True)),
 ([100700871, 900235], (0.525, True)),
 ([900266, 100700871], (0.275, True)),
 ([900162, 900183, 900235, 100700871, 22004601, 635719], (0.275, True)),
 ([100700871, 900235], (0.525, True)),
 ([100700871], (0.3, True)),
 ([100700871, 900235], (0.525, True)),
 ([100700871, 900142], (0.225, True)),
 ([900235, 100700871, 900226, 900235, 900246], (0.6, True)),
 ([900233, 22010060, 900149, 900233, 900233, 900149, 100700871], (0.2, True)),
 ([90023

بنظر می رسد این روش هر پنج مسیر پرتکرار را یافته است

بررسی تاثیر افزایش هایپرپلین: با افزایش تعداد هایپر پلین ها، دقت به صورت قابل ملاحظه ای بهتر شده است. در واقع با افزایش هایپرپلین ها مسیرها یی که شباهت کمتری دارند و در حالت قبل تصادفا درجواب ها حاضر شده اند، هرس می شوند و در خروجی نهایی بشتر مسیرهایی می آیند که واقعا به مسیر اصلی شبیه بوده و شبیه تر به خروجی واقعی شباهت کسینوسی بین دو مسیر است. در واقع با افزایش تعداد هایپرپلین ها، به قانون اعداد بزرگ نزدیک می شویم و خروجی روش مین هشینگ به فاصله کسینوسی (که هدف اصلی طراحی این روش هم بود) نزدیک میشویم.

اما قابل توجه است که این روش کمی برای مسیرهایی که طول بیشتری دارند، برخلاف روش ارزیابی شباهت بر مبنای فاصله کسینوسی ، بایاس دارد زیرا هر چه طول مسیر بیشتر باشد احتمال دارا بودن نقاط مسیر کویری هم بیشتر است و در این جا چون با طول مسیرها شباهت ها را نرمال نمیکنیم و تنها حالت باینری است، پس مسیرهای طولانی تر با احتمال بیشتری در خروجی ظاهر میشوند